### Подготовка датасета по показателю количество полученных яиц

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

from pylab import rcParams
from IPython.display import display
import math
from prophet import Prophet
pd.set_option('display.max_columns', 130)


import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter("ignore", category=InterpolationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)



In [2]:
df = pd.read_excel("../../Data cleansing/output data/Просуммированные по категориям с доп регрессорами.xlsx")
df.head(5)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Верблюды,АКМОЛИНСКАЯ ОБЛАСТЬ,0.00,0.00,0.40,0.00,0.00,0.00,0.09,1.00,0.00,0.00,0.20,0.00,0.00,0.18,0.28,0.00,0.00,0.40,0.00,0.00,0.65,0.00,0.31,1.00,0.00,0.00,0.00,0.00,0.00,2.01,0.00,1.20,0.00,0.00,2.18,0.39,0.00,0.00,1.04,0.00,0.14,2.08,0.00,0.00,0.00,0.00,0.00,0.30,0.00,0.00,0.00,0.66,0.00,0.33,0.00,0.9,0.00,0.00,0.00,10.08,0.00,0.0,0.00,0.0,0.00,0.54,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.36,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00
1,Верблюды,АКТЮБИНСКАЯ ОБЛАСТЬ,101.98,67.47,374.84,115.59,218.72,14.15,19.77,3.00,39.16,46.16,238.56,463.35,109.04,72.94,384.96,114.35,221.89,10.93,18.79,3.50,38.53,46.64,216.08,472.07,110.32,68.17,380.15,127.12,217.61,14.98,21.26,7.14,44.63,51.78,239.43,513.33,115.70,69.40,371.53,130.17,218.41,14.59,21.76,7.19,45.97,55.48,252.68,527.66,117.94,70.37,385.75,117.80,218.45,15.54,21.78,7.2,47.06,58.02,257.54,543.35,119.80,71.3,396.90,121.8,228.10,15.90,21.9,7.2,63.00,59.90,260.20,552.10,121.60,71.8,398.40,128.50,228.80,13.10,22.50,7.20,63.90,61.90,150.70,554.90,121.30,73.3,404.20,128.70,234.90,13.60,23.60,6.3,49.80,62.4,153.70,552.8,118.07,69.17,384.21,121.01,223.46,11.76,21.39,5.52,48.18,61.39,150.87,531.79,119.50,72.60,391.0,126.1,229.10,12.2,24.10,5.7,49.87,63.00,154.3,538.40
2,Верблюды,АЛМАТИНСКАЯ ОБЛАСТЬ,1.00,0.20,51.60,25.40,0.00,61.90,76.87,95.57,16.09,0.10,10.80,46.78,15.06,13.70,131.80,9.20,17.89,130.12,2.00,0.00,4.99,2.06,4.90,44.48,15.06,23.94,41.60,19.54,3.00,15.14,18.50,14.82,14.52,12.60,9.92,43.11,20.72,2.96,15.24,0.00,1.00,20.03,3.44,7.77,117.25,1.00,0.00,20.34,21.95,3.04,17.91,26.81,9.16,18.80,13.37,0.0,17.29,15.90,4.69,23.10,10.85,9.7,42.75,0.0,6.21,21.10,4.0,2.6,28.10,4.53,79.50,93.18,23.70,11.2,16.00,2.00,3.10,19.62,2.30,4.85,11.18,2.80,9.65,36.44,10.60,11.5,26.10,7.20,12.75,11.20,11.20,27.0,25.26,17.9,21.90,12.5,16.55,16.81,22.78,12.10,12.44,6.85,39.65,17.22,8.85,27.48,0.48,29.72,18.90,17.40,12.5,16.4,11.60,16.7,15.70,24.0,7.70,6.90,5.4,12.60
3,Верблюды,АТЫРАУСКАЯ ОБЛАСТЬ,213.89,167.70,306.60,164.97,342.57,192.00,43.60,113.03,262.37,193.97,308.17,1087.33,325.10,190.60,301.10,154.84,328.50,220.30,66.10,118.20,234.90,196.51,270.92,974.18,303.30,167.41,323.54,182.34,349.10,249.40,57.57,88.40,221.21,199.63,278.80,964.02,305.22,159.62,326.87,159.50,367.20,257.50,60.01,78.10,253.27,332.00,355.05,989.86,280.08,165.05,334.51,134.85,367.97,321.22,126.22,93.7,263.10,366.66,348.54,1041.38,293.40,154.8,339.44,143.6,405.80,308.35,88.5,130.5,575.30,458.70,357.62,998.10,292.76,188.3,487.02,129.28,421.44,351.20,95.77,97.54,624.80,631.65,499.30,1179.66,286.67,189.8,513.16,171.46,402.10,432.22,121.53,126.2,637.50,651.7,482.94,1412.2,307.85,200.59,466.60,182.63,414.56,405.19,184.80,154.96,717.92,719.01,522.83,1164.77,323.81,423.76,441.9,219.1,487.69,446.9,191.43,133.5,730.34,709.68,537.6,930.28
4,Верблюды,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0.30,0.00,0.00,1.14,6.12,4.70

In [3]:
df['Показатель'].unique()

array(['Верблюды', 'КРС', 'Лошади', 'Молоко', 'Овцы и козы', 'Птица',
       'Свиньи', 'Яйца', 'Температура', 'Поголовье: КРС',
       'Поголовье: лошади', 'Поголовье: овцы и козы', 'Поголовье: свиньи',
       'Поголовье: птица домашняя', 'Поголовье: верблюды', 'Осадки',
       'Цена: Говядина', 'Цена: Баранина', 'Цена: Молоко', 'Цена: Яйца'],
      dtype=object)

In [4]:
df_eggs = df[df['Показатель'].isin(['Яйца', 'Температура', 'Осадки', 'Поголовье: птица домашняя', 'Цена: Яйца'])]
df_eggs.sample(10)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
330,Осадки,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.500000,0.000000,0.000000,0.400000,18.500000,72.20,19.800000,29.400000,22.100000,35.100000,35.600000,8.600000,6.800000,1.800000,2.600000,0.000000,6.300000,5.000000,36.900000,25.100000,39.200000,1.000000,24.000000,26.100000,0.500000,0.300000,3.600000,2.600000,0.400000,17.100000,1.000000,28.300000,18.400000,65.500000,18.900000,8.000000,2.500000,0.000000,0.000000,1.550000,7.800000,3.900000,25.000000,55.400000,18.300000,37.600000,18.600000,34.900000,1.500000,0.000000,2.600000,1.550000,18.400000,54.400000,8.100000,27.100000,24.100000,15.700000,19.400000,17.500000,0.000000,0.900000,10.600000,0.400000,23.300000,34.900000,20.500000,2.660000e+01,8.600000e+00,46.900000,3.290000e+01,32.400000,1.500000,12.200000,0.000000e+00,0.500000,13.600000,1.210000e+01,1.730000e+01
169,Температура,АКТЮБИНСКАЯ ОБЛАСТЬ,-13.483871,-11.821429,-5.990323,6.356667,15.609677,25.126667,23.167742,20.732258,16.576667,4.187097,-2.920000,-3.845161,-12.474194,-3.806897,-1.054839,9.783333,16.167742,19.766667,22.332258,24.861290,13.863333,3.980645,-4.056667,-10.980645,-10.119355,-12.132143,-5.500000,5.953333,14.645161,19.843333,23.587097,23.758065,15.55,5.164516,1.133333,-10.583871,-15.477419,-11.739286,-7.812903,6.746667,16.445161,18.810000,25.893548,20.570968,15.266667,6.838710,-4.47,-9.922581,-12.296774,-11.685714,-0.583871,8.230000,17.806452,22.266667,24.580645,19.545161,10.923333,8.012903,-4.690000,-7.016129,-5.767742,-5.320690,0.874194,8.100000,17.722581,21.006667,25.261290,20.774194,13.356667,6.409677,-4.913333,-13.645161,-11.145161,-10.978571,-6.683871,7.923333,20.609677,24.920000,23.458065,24.222581,12.753333,5.693548,-3.143333,-6.532258,-9.961290,-5.875000,-6.783871,10.880000,13.245161,20.303333,22.854839,23.264516,15.700000,6.512903,-2.326667,-11.690323,-15.625806,-10.800000,1.522581,10.190000,17.077419,21.763333,24.461290,22.083871,14.703333,7.032258,2.976667,-7.170968,-1.120645e+01,-1.156897e+01,-4.800000,1.351333e+01,13.083871,22.893333,22.912903,2.020645e+01,14.083333,6.667742,-1.520000e+00,-6.419355e+00
178,Температура,ПАВЛОДАРСКАЯ ОБЛАСТЬ,-13.822581,-12.507143,-6.219355,7.010000,15.622581,21.330000,21.487097,18.522581,11.726667,4.558065,-8.146667,-5.448387,-21.106452,-9.168966,-1.925806,10.373333,13.654839,20.090000,21.196774,19.154839,15.370000,-0.364516,-11.570000,-10.822581,-11.709677,-14.510714,-7.693548,7.443333,14.783871,21.256667,20.561290,19.264516,11.72,3.464516,-3.220000,-13.458065,-22.641935,-16.282143,-6.487097,5.953333,9.958065,19.356667,20.525806,18.374194,11.623333,4.996774,-5.43,-17.070968,-16.467742,-15.671429,-2.877419,6.056667,12.241935,17.103333,21.961290,20.670968,12.133333,6.967742,-8.313333,-8.512903,-10.296774,-7.862069,-2.141935,11.133333,18.625806,19.936667,22.303226,20.364516,12.070000,4.683871,-5.173333,-16.141935,-20.022581,-13.792857,-5.874194,6.513333,17.470968,18.2

In [5]:
# Step 1: Pivot to wide format (each indicator becomes columns of periods)
df_wide = df_eggs.pivot(index="Регион", columns="Показатель")

# Step 2: Flatten multi-level columns: ('2015-01', 'КРС') → 'КРС_2015-01'
df_wide.columns = [f"{col[1]}_{col[0]}" for col in df_wide.columns]
df_wide = df_wide.reset_index()

# Step 3: Melt: one row per region-period-indicator
df_melted = df_wide.melt(id_vars="Регион", var_name="indicator_period", value_name="value")

# Step 4: Extract 'Период' and 'Показатель' from the combined column
df_melted["Период"] = df_melted["indicator_period"].str.extract(r"_(\d{4}-\d{2})$")
df_melted["Показатель"] = df_melted["indicator_period"].str.extract(r"^(.+)_\d{4}-\d{2}")

# Step 5: Pivot again to get final modeling format: one row per region+period, one column per indicator
df_eggs = df_melted.pivot_table(index=["Регион", "Период"], columns="Показатель", values="value").reset_index()
print(df_eggs.groupby("Регион").size().reset_index(name="Количество строк"))
df_eggs

                            Регион  Количество строк
0              АКМОЛИНСКАЯ ОБЛАСТЬ               120
1              АКТЮБИНСКАЯ ОБЛАСТЬ               120
2              АЛМАТИНСКАЯ ОБЛАСТЬ               120
3               АТЫРАУСКАЯ ОБЛАСТЬ               120
4   ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ               120
5                          ГАЛМАТЫ               120
6                          ГАСТАНА               120
7                         ГШЫМКЕНТ                79
8               ЖАМБЫЛСКАЯ ОБЛАСТЬ               120
9    ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ               120
10          КАРАГАНДИНСКАЯ ОБЛАСТЬ               120
11            КОСТАНАЙСКАЯ ОБЛАСТЬ               120
12          КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ               120
13           МАНГИСТАУСКАЯ ОБЛАСТЬ               120
14                    ОБЛАСТЬ АБАЙ                31
15                  ОБЛАСТЬ ЖЕТІСУ                31
16                  ОБЛАСТЬ ҰЛЫТАУ                31
17            ПАВЛОДАРСКАЯ ОБЛАСТЬ            

Показатель,Регион,Период,Осадки,Поголовье: птица домашняя,Температура,Цена: Яйца,Яйца
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-01,9.8,4436655.0,-12.490323,100.600000,51667.7
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-02,9.8,4446764.0,-10.192857,88.829800,49357.2
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-03,8.3,4461524.0,-5.870968,87.053204,59105.8
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-04,8.8,4575245.0,4.490000,87.227310,61953.7
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-05,42.8,4778690.0,14.574194,69.607394,72700.2
...,...,...,...,...,...,...,...
2166,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,0.0,2167498.0,27.874194,483.016761,23808.0
2167,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,0.5,2092996.0,20.766667,489.295979,22847.1
2168,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-10,13.6,2088086.0,13.200000,530.396841,21639.9
2169,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-11,12.1,2030035.0,6.500000,546.839143,19659.4


In [6]:
df_eggs = df_eggs[df_eggs["Регион"] != 'РЕСПУБЛИКА КАЗАХСТАН']
df_eggs = df_eggs.drop(columns=['Осадки', 'Поголовье: птица домашняя', 'Температура', 'Цена: Яйца'])
df_eggs.sample(10)

Показатель,Регион,Период,Яйца
1008,ЖАМБЫЛСКАЯ ОБЛАСТЬ,2022-06,13880.4
1451,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2019-05,754.0
744,ГАСТАНА,2017-01,8.0
1589,МАНГИСТАУСКАЯ ОБЛАСТЬ,2020-11,53.2
347,АЛМАТИНСКАЯ ОБЛАСТЬ,2023-12,46274.6
789,ГАСТАНА,2020-10,0.1
1437,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2018-03,393.8
767,ГАСТАНА,2018-12,3.4
433,АТЫРАУСКАЯ ОБЛАСТЬ,2021-02,3669.0
1280,КОСТАНАЙСКАЯ ОБЛАСТЬ,2015-02,33173.6


In [7]:
df_eggs.to_excel("Датасет по яйцам.xlsx", index=False)